In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# ตอนนี้ notebook อยู่ใน .../rossmann/notebooks
# parent = .../rossmann  (โฟลเดอร์หลักที่มีไฟล์ train/test/store/sample_submission)
PROJECT_DIR = Path.cwd().parent
RAW_DIR = PROJECT_DIR  # ✅ เพราะไฟล์อยู่ใน rossmann ตามรูป
ARTIFACTS_DIR = PROJECT_DIR / "artifacts"
DATA_DIR = PROJECT_DIR / "data"

ARTIFACTS_DIR.mkdir(exist_ok=True)
DATA_DIR.mkdir(exist_ok=True)

print("PROJECT_DIR:", PROJECT_DIR)
print("Files in RAW_DIR:", [p.name for p in RAW_DIR.glob("*")])


PROJECT_DIR: C:\Users\USER\Desktop\rossman
Files in RAW_DIR: ['.ipynb_checkpoints', 'artifacts', 'data', 'images', 'notebooks', 'sample_submission.csv', 'store.csv', 'submission', 'test.csv', 'train.csv']


In [2]:
print("All files with extensions:", [p.name for p in RAW_DIR.glob("*.*")])


All files with extensions: ['.ipynb_checkpoints', 'sample_submission.csv', 'store.csv', 'test.csv', 'train.csv']


In [3]:
train = pd.read_csv(RAW_DIR / "train.csv", low_memory=False)
test  = pd.read_csv(RAW_DIR / "test.csv", low_memory=False)
store = pd.read_csv(RAW_DIR / "store.csv", low_memory=False)

In [4]:
train = pd.read_csv(RAW_DIR / "train.csv", dtype={"StateHoliday": "string"}, low_memory=False)
test  = pd.read_csv(RAW_DIR / "test.csv", dtype={"StateHoliday": "string"}, low_memory=False)
store = pd.read_csv(RAW_DIR / "store.csv", low_memory=False)

In [5]:
train.dtypes


Store                     int64
DayOfWeek                 int64
Date                     object
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday     string[python]
SchoolHoliday             int64
dtype: object

In [6]:
train["StateHoliday"].unique()[:10]


<StringArray>
['0', 'a', 'b', 'c']
Length: 4, dtype: string

In [7]:
def summarize_df(df):
    s = pd.DataFrame({
        "column": df.columns,
        "dtype": df.dtypes.astype(str).values,
        "missing_count": df.isna().sum().values,
        "missing_pct": (df.isna().mean() * 100).round(2).values,
        "n_unique": df.nunique(dropna=True).values
    }).sort_values("missing_count", ascending=False).reset_index(drop=True)
    return s

train_summary = summarize_df(train)
test_summary  = summarize_df(test)
store_summary = summarize_df(store)

train_summary.head(20)


,column,dtype,missing_count,missing_pct,n_unique
0,Store,int64,0,0.0,1115
1,DayOfWeek,int64,0,0.0,7
2,Date,object,0,0.0,942
3,Sales,int64,0,0.0,21734
4,Customers,int64,0,0.0,4086
5,Open,int64,0,0.0,2
6,Promo,int64,0,0.0,2
7,StateHoliday,string,0,0.0,4
8,SchoolHoliday,int64,0,0.0,2


In [8]:
train_summary.to_csv(ARTIFACTS_DIR / "train_missing_dtype_summary.csv", index=False)
test_summary.to_csv(ARTIFACTS_DIR / "test_missing_dtype_summary.csv", index=False)
store_summary.to_csv(ARTIFACTS_DIR / "store_missing_dtype_summary.csv", index=False)

print("Saved summaries to:", ARTIFACTS_DIR)


Saved summaries to: C:\Users\USER\Desktop\rossman\artifacts


In [9]:
train["Date"] = pd.to_datetime(train["Date"])
test["Date"]  = pd.to_datetime(test["Date"])

print(train["Date"].dtype, test["Date"].dtype)


datetime64[ns] datetime64[ns]


In [10]:
train_merged = train.merge(store, on="Store", how="left")
test_merged  = test.merge(store, on="Store", how="left")

print("train_merged:", train_merged.shape)
print("test_merged:", test_merged.shape)

train_merged.head()


train_merged: (1017209, 18)
test_merged: (41088, 17)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [11]:
print("Missing StoreType in train_merged:", train_merged["StoreType"].isna().sum())
print("Missing StoreType in test_merged:", test_merged["StoreType"].isna().sum())


Missing StoreType in train_merged: 0
Missing StoreType in test_merged: 0


In [12]:
def basic_impute(df):
    df = df.copy()

    # CompetitionDistance เติม median + สร้าง flag
    if "CompetitionDistance" in df.columns:
        df["CompetitionDistance_missing"] = df["CompetitionDistance"].isna().astype(int)
        df["CompetitionDistance"] = df["CompetitionDistance"].fillna(df["CompetitionDistance"].median())

    # เติม 0 ให้คอลัมน์เกี่ยวกับปี/เดือน/สัปดาห์ + สร้าง flag
    for col in ["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", "Promo2SinceWeek", "Promo2SinceYear"]:
        if col in df.columns:
            df[f"{col}_missing"] = df[col].isna().astype(int)
            df[col] = df[col].fillna(0).astype(int)

    # PromoInterval เติม "None" + flag
    if "PromoInterval" in df.columns:
        df["PromoInterval_missing"] = df["PromoInterval"].isna().astype(int)
        df["PromoInterval"] = df["PromoInterval"].fillna("None")

    return df

train_clean = basic_impute(train_merged)
test_clean  = basic_impute(test_merged)

summarize_df(train_clean).head(20)


,column,dtype,missing_count,missing_pct,n_unique
0,Store,int64,0,0.0,1115
1,DayOfWeek,int64,0,0.0,7
2,Promo2SinceYear_missing,int32,0,0.0,2
3,Promo2SinceWeek_missing,int32,0,0.0,2
4,CompetitionOpenSinceYear_missing,int32,0,0.0,2
5,CompetitionOpenSinceMonth_missing,int32,0,0.0,2
6,CompetitionDistance_missing,int32,0,0.0,2
7,PromoInterval,object,0,0.0,4
8,Promo2SinceYear,int32,0,0.0,8
9,Promo2SinceWeek,int32,0,0.0,25


In [13]:
train_clean.to_csv(DATA_DIR / "train_processed.csv", index=False)
test_clean.to_csv(DATA_DIR / "test_processed.csv", index=False)

print("Saved processed files to:", DATA_DIR)


Saved processed files to: C:\Users\USER\Desktop\rossman\data


In [14]:
print("Processed train columns:", train_clean.shape)
print("Missing after clean (top 10):")
print(summarize_df(train_clean).head(10))


Processed train columns: (1017209, 24)
Missing after clean (top 10):
                              column   dtype  missing_count  missing_pct  \
0                              Store   int64              0          0.0   
1                          DayOfWeek   int64              0          0.0   
2            Promo2SinceYear_missing   int32              0          0.0   
3            Promo2SinceWeek_missing   int32              0          0.0   
4   CompetitionOpenSinceYear_missing   int32              0          0.0   
5  CompetitionOpenSinceMonth_missing   int32              0          0.0   
6        CompetitionDistance_missing   int32              0          0.0   
7                      PromoInterval  object              0          0.0   
8                    Promo2SinceYear   int32              0          0.0   
9                    Promo2SinceWeek   int32              0          0.0   

   n_unique  
0      1115  
1         7  
2         2  
3         2  
4         2  
5         